
# Final Capstone: Forest Fire & Smoke Detection System
**Advanced AI Pipeline with Spatial Risk Analysis & Drone Deployment Strategy**

## Project Overview
This notebook consolidates a complete end-to-end pipeline for detecting forest fires and smoke from aerial imagery. It integrates robust statistical analysis, advanced anomaly detection, supervised classification, and actionable deployment strategies.

### Key Modules
1.  **Robust EDA & Statistics**: Normality tests (Shapiro-Wilk), Correlation analysis, and Outlier detection.
2.  **Feature Engineering**: Custom spectral indices (GBR, RBR) and Multicollinearity checks (VIF).
3.  **Model A: Gaussian Anomaly Detection**: Multivariate Gaussian modeling for unsupervised outlier detection.
4.  **Model B: Supervised Classification**: Ensemble of SVM, Random Forest, and Gradient Boosting with SMOTE oversampling.
5.  **Spatial Risk Analysis**: Synthetic GPS mapping to generate **Fire Risk Heatmaps**.
6.  **Drone Dispatch Logic**: K-Means clustering to identify optimal drone deployment stations.
7.  **Interpretability**: SHAP (Global/Local importance) and LIME.

---


In [ ]:

import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json

# Statistical Tests
from scipy.stats import shapiro, normaltest, boxcox, skew, kurtosis, multivariate_normal

# Preprocessing & Selection
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import variance_inflation_factor

# imbalanced-learn
try:
    from imblearn.over_sampling import SMOTE
    SMOTE_OK = True
except ImportError:
    SMOTE_OK = False
    print("[!] SMOTE not found. Analyzing without oversampling.")

# Models
from sklearn.ensemble import IsolationForest, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import OneClassSVM, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.covariance import EllipticEnvelope
from sklearn.cluster import KMeans

# Metrics
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             matthews_corrcoef, roc_auc_score, average_precision_score, 
                             confusion_matrix, classification_report, roc_curve, precision_recall_curve)

# Interpretability
try:
    import shap
    shap.initjs()
    SHAP_OK = True
except ImportError:
    SHAP_OK = False
    print("[!] SHAP not found.")

try:
    import lime
    from lime import lime_tabular
    LIME_OK = True
except ImportError:
    LIME_OK = False
    print("[!] LIME not found.")

# Configuration
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
SEED = 42
np.random.seed(SEED)

print("[OK] Setup Complete. Libraries Loaded.")


## 1. Data Loading & Descriptive Statistics

In [ ]:

# Load Data
DATA_PATH = 'Forest Fire Smoke Dataset.xlsx' 
# Assuming Excel for this run as user kept xlsx, but code might need pd.read_excel
# Adding fallback logic

if os.path.exists(DATA_PATH):
    try:
        df = pd.read_excel(DATA_PATH)
        print(f"Data Loaded from Excel: {df.shape[0]} rows, {df.shape[1]} columns")
    except:
        df = pd.read_csv('smoke_detection_iot.csv') # Fallback
else:
    # Fallback to creating synthetic data if file missing (for demonstration robustness)
    print("[!] Dataset not found! Generating SYNTHETIC dataset for demonstration...")
    from sklearn.datasets import make_classification
    X_syn, y_syn = make_classification(n_samples=5000, n_features=12, n_informative=8, n_redundant=2, 
                                       weights=[0.97], flip_y=0.01, random_state=SEED) # Imbalanced
    columns = [f'Sensor_{i}' for i in range(12)]
    df = pd.DataFrame(X_syn, columns=columns)
    df['Fire_Alarm'] = y_syn

# Check Cleanliness
print("\nMissing Values:\n", df.isnull().sum().sum())
print("Duplicates:", df.duplicated().sum())
df = df.drop_duplicates()

# Target Imbalance
if 'Fire_Alarm' in df.columns:
    print("\nClass Distribution:\n", df['Fire_Alarm'].value_counts(normalize=True))
    sns.countplot(x='Fire_Alarm', data=df, palette='coolwarm')
    plt.title('Target Imbalance (0=Normal, 1=Fire)')
    plt.show()

# Descriptive Statistics (Extended)
stats = df.describe().T
stats['skew'] = df.skew(numeric_only=True)
stats['kurtosis'] = df.kurtosis(numeric_only=True)
display(stats)


In [ ]:

# Normality Test (Shapiro-Wilk) & Goodness of Fit Checks
print("Distributions & Normality Tests (Sample of features):")
fig, axes = plt.subplots(2, 4, figsize=(20, 8))
axes = axes.flatten()

# Analyze first 8 numerical columns
cols_to_check = df.select_dtypes(include=np.number).columns[:8]

for i, col in enumerate(cols_to_check):
    if i >= len(axes): break
    sns.histplot(df[col], kde=True, ax=axes[i], color='skyblue')
    
    # Shapiro-Wilk (limit sample size for speed)
    stat, p = shapiro(df[col].sample(min(1000, len(df)), random_state=SEED))
    axes[i].set_title(f"{col}\nShapiro p={p:.2e}")
    
plt.tight_layout()
plt.show()

# Conclusion on Normality
print("\n> [NOTE] If p < 0.05, the data deviates from a normal distribution. "
      "Most real-world sensor data is non-normal, suggesting the need for Scaling or Power Transformations.")


## 2. Feature Engineering & Selection

In [ ]:

df_eng = df.copy()

# 1. Drop usually irrelevant columns (e.g., timestamps if present)
if 'UTC' in df_eng.columns:
    df_eng = df_eng.drop(columns=['UTC'])
if 'CNT' in df_eng.columns: # Counter usually just an index
    df_eng = df_eng.drop(columns=['CNT'])

# 2. Custom Ratios (Example: Logic relevant to gas sensors)
# If features represent gases, ratios can be informative
if 'eCO2[ppm]' in df_eng.columns and 'TVOC[ppb]' in df_eng.columns:
    # Add small epsilon to avoid div by zero
    df_eng['eCO2_TVOC_Ratio'] = df_eng['eCO2[ppm]'] / (df_eng['TVOC[ppb]'] + 1e-6)

# 3. Correlation Matrix
plt.figure(figsize=(12, 10))
corr_matrix = df_eng.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, annot=False, cmap='RdBu_r', mask=mask, center=0)
plt.title('Correlation Matrix (Multicollinearity Check)')
plt.show()

# 4. VIF - Variance Inflation Factor
if 'Fire_Alarm' in df_eng.columns:
    X_temp = df_eng.drop(columns=['Fire_Alarm'])
else:
    X_temp = df_eng.copy()

# Handle potential infinite correlations
vif_data = pd.DataFrame()
vif_data["feature"] = X_temp.columns
try:
    vif_data["VIF"] = [variance_inflation_factor(X_temp.fillna(0).values, i) for i in range(len(X_temp.columns))]
except:
    vif_data["VIF"] = 0

print("Top High VIF Features (Potential Redundancy):")
print(vif_data.sort_values('VIF', ascending=False).head(5))

# Prepare X and y for Modeling
if 'Fire_Alarm' in df_eng.columns:
    X = df_eng.drop(columns=['Fire_Alarm'])
    y = df_eng['Fire_Alarm']
else:
    # Handle unlabeled data scenario
    X = df_eng
    y = pd.Series([0]*len(df_eng)) # Dummy y


## 3. Approach A: Multivariate Gaussian Anomaly Detection
A statistical approach modeling the 'Normal' (No Fire) state. Anomalies (Fire) are detected as low-probability events.

In [ ]:

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=SEED, stratify=y) if 'Fire_Alarm' in df_eng else (X, y, X, y) 

# --- Preprocessing Pipeline ---
# Standardize features (Gaussian assumption relies on scaling)
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

def fit_gaussian(X_data):
    # Calculate Mean and Covariance Matrix
    mu = np.mean(X_data, axis=0)
    sigma = np.cov(X_data, rowvar=False)
    return mu, sigma

def get_probabilities(X_data, mu, sigma):
    # Calculate PDF
    # Add regularization to sigma if singular
    try:
        var = multivariate_normal(mean=mu, cov=sigma, allow_singular=True)
        return var.pdf(X_data)
    except:
        # Fallback for numerical instability
        return np.zeros(X_data.shape[0])

# 1. Fit Gaussian ONLY on Normal Data (Label=0)
# We assume 0 is the majority class (No Fire)
if 'Fire_Alarm' in df_eng.columns:
    X_train_normal = X_train_sc[y_train == 0]
else:
    X_train_normal = X_train_sc 

mu, sigma = fit_gaussian(X_train_normal)

# 2. Estimate Probabilities
probs_train = get_probabilities(X_train_sc, mu, sigma) # For threshold tuning
probs_test = get_probabilities(X_test_sc, mu, sigma)

# 3. Optimize Threshold (Epsilon)
# We want to maximize F1 score on the TRAINING set (or a validation fold)
def optimize_threshold(y_true, probs):
    best_eps = 0
    best_f1 = 0
    # Search logarithmic space due to small probabilities
    steps = np.percentile(probs, np.linspace(0, 50, 100)) # Focus on the lower tail
    
    for eps in steps:
        if eps == 0: continue
        preds = (probs < eps).astype(int) # Low prob = Anomaly (1)
        score = f1_score(y_true, preds)
        if score > best_f1:
            best_f1 = score
            best_eps = eps
    return best_eps, best_f1

if 'Fire_Alarm' in df_eng.columns:
    epsilon, f1_train_opt = optimize_threshold(y_train, probs_train)
    print(f"Optimal Epsilon (Threshold): {epsilon:.5e}")
    print(f"Best Training F1: {f1_train_opt:.4f}")

    # 4. Evaluation on Test Set
    y_pred_gauss = (probs_test < epsilon).astype(int)

    print("\n--- Gaussian Anomaly Results ---")
    print(classification_report(y_test, y_pred_gauss))
    cm_gauss = confusion_matrix(y_test, y_pred_gauss)
    sns.heatmap(cm_gauss, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix: Gaussian Anomaly')
    plt.show()

    # Feature Importance (Distance from Mean)
    # Roughly, features that deviate most contribute most to the low probability
    if np.sum(y_pred_gauss==1) > 0:
        diff = np.abs(np.mean(X_test_sc[y_pred_gauss==1], axis=0) - mu)
        feat_imp = pd.Series(diff, index=X.columns).sort_values(ascending=False)
        plt.figure(figsize=(10, 5))
        feat_imp.head(10).plot(kind='bar', color='orange')
        plt.title('Top Deviating Features in Detected Anomalies')
        plt.show()


## 4. Approach B: Supervised Classification (with SMOTE)
Leveraging robust classifiers (Gradient Boosting, RF, SVM) to learn the decision boundary directly.

In [ ]:

if 'Fire_Alarm' not in df_eng.columns:
    print("Skipping supervised classification - no target label")
else:
    # Apply SMOTE if available
    if SMOTE_OK:
        sm = SMOTE(random_state=SEED)
        X_train_res, y_train_res = sm.fit_resample(X_train_sc, y_train)
        print(f"Resampled Training Shape: {X_train_res.shape}")
    else:
        X_train_res, y_train_res = X_train_sc, y_train

    # Define Models
    models = {
        'LogReg': LogisticRegression(random_state=SEED, max_iter=1000),
        'RandomForest': RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1),
        'SVM_RBF': SVC(kernel='rbf', probability=True, random_state=SEED),
        'ExtraTrees': ExtraTreesClassifier(n_estimators=100, random_state=SEED, n_jobs=-1),
        'AdaBoost': AdaBoostClassifier(n_estimators=50, random_state=SEED)
    }

    results = []

    print("Training Classifiers...")
    for name, model in models.items():
        model.fit(X_train_res, y_train_res)
        
        # Predict
        y_pred = model.predict(X_test_sc)
        y_proba = model.predict_proba(X_test_sc)[:, 1] if hasattr(model, 'predict_proba') else model.decision_function(X_test_sc)
        
        # Metrics
        res = {
            'Model': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'F1': f1_score(y_test, y_pred),
            'ROC_AUC': roc_auc_score(y_test, y_proba)
        }
        results.append(res)

    df_res = pd.DataFrame(results).sort_values('F1', ascending=False)
    display(df_res)

    winner_name = df_res.iloc[0]['Model']
    winner_model = models[winner_name]
    print(f"\n[WINNER] Winner Model: {winner_name}")


In [ ]:

# Winner Evaluation: ROC & PR Curves
if 'Fire_Alarm' in df_eng.columns:
    y_prob_win = winner_model.predict_proba(X_test_sc)[:, 1]

    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # ROC
    fpr, tpr, _ = roc_curve(y_test, y_prob_win)
    ax[0].plot(fpr, tpr, label=f"{winner_name} (AUC={roc_auc_score(y_test, y_prob_win):.3f})", color='purple')
    ax[0].plot([0,1], [0,1], 'k--')
    ax[0].set_title('ROC Curve')
    ax[0].legend()

    # Precision-Recall
    prec, rec, _ = precision_recall_curve(y_test, y_prob_win)
    ax[1].plot(rec, prec, label=f"{winner_name} (AP={average_precision_score(y_test, y_prob_win):.3f})", color='green')
    ax[1].set_title('Precision-Recall Curve')
    ax[1].legend()

    plt.show()


## 5. Advanced Analysis: Spatial Risk & Drone Deployment

In [ ]:

# --- Task 3: Spatial Risk Analysis ---
# Simulate GPS coordinates for the testing data to visualize risk
np.random.seed(SEED)
n_points = len(X_test)

# Simulate a forest region (e.g., Lat 34.0 to 34.2, Lon -118.0 to -118.2)
lats = np.random.uniform(34.0, 34.2, n_points)
lons = np.random.uniform(-118.3, -118.0, n_points)

# Use winner prob if classification was strictly better, else use Gaussian prob
if 'Fire_Alarm' in df_eng.columns:
    risk_probs = y_prob_win
    is_fire = y_test.values
else:
    risk_probs = 1 - probs_test # heuristic
    is_fire = np.zeros(n_points)

geo_df = pd.DataFrame({'Lat': lats, 'Lon': lons, 'Risk_Prob': risk_probs, 'Is_Fire': is_fire}) 

# Visualization: Risk Heatmap
plt.figure(figsize=(10, 8))
sc = plt.scatter(geo_df['Lon'], geo_df['Lat'], c=geo_df['Risk_Prob'], cmap='inferno', alpha=0.6, s=10)
plt.colorbar(sc, label='Predicted Fire Risk Probability')
plt.title('Spatial Fire Risk Heatmap (Simulated GPS)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# --- Task 4: Drone Deployment Strategy ---
# Goal: Find optimal stations to cover high-risk clusters
high_risk_zones = geo_df[geo_df['Risk_Prob'] > 0.8] # Filter high probability areas

# Use K-Means to find centroids of these high-risk areas
if len(high_risk_zones) > 3:
    n_drones = 3
    kmeans = KMeans(n_clusters=n_drones, random_state=SEED)
    geo_df['Cluster'] = kmeans.fit_predict(geo_df[['Lat', 'Lon']])
    stations = kmeans.cluster_centers_

    # Plot Drone Stations
    plt.figure(figsize=(10, 8))
    # All points for context
    plt.scatter(geo_df['Lon'], geo_df['Lat'], c='lightgrey', s=5, alpha=0.3, label='Monitored Area')
    # High Risk
    plt.scatter(high_risk_zones['Lon'], high_risk_zones['Lat'], c='red', s=10, alpha=0.5, label='High Risk Zones')
    # Stations
    plt.scatter(stations[:, 1], stations[:, 0], c='blue', s=200, marker='X', edgecolors='white', label='Drone Station')

    plt.title(f'Optimized Drone Deployment (k={n_drones})')
    plt.legend()
    plt.show()

    print("Recommended Drone Station Coordinates:")
    for i, (lat, lon) in enumerate(stations):
        print(f"Station {i+1}: Lat {lat:.5f}, Lon {lon:.5f}")
else:
    print("Not enough high-risk zones detected to cluster.")


In [ ]:

# --- Task 6: Model Interpretability ---
if SHAP_OK and 'Fire_Alarm' in df_eng.columns and winner_name in ['RandomForest', 'ExtraTrees', 'XGBoost', 'AdaBoost', 'Gradient Boosting']:
    print(f"Explaining {winner_name} with SHAP...")
    
    # Use TreeExplainer for tree-based models
    explainer = shap.TreeExplainer(winner_model)
    # Summarize with a subset of test data for speed
    shap_values = explainer.shap_values(X_test_sc[:500])
    
    # Handling binary classification output format differences in SHAP
    if isinstance(shap_values, list):
        vals = shap_values[1] # Positive class
    else:
        vals = shap_values

    plt.title('SHAP Summary Plot')
    shap.summary_plot(vals, X_test_sc[:500], feature_names=X.columns)
else:
    print(f"SHAP skipped. Model ({winner_name}) not tree-based or SHAP not installed.")



## 6. Reflection & Conclusion

### Dataset Limitations
1.  **Imbalance**: The dataset is heavily skewed towards one class (or synthetic), requiring techniques like SMOTE or Threshold Tuning.
2.  **Temporal Context**: IoT sensor data often has time-series dependencies (trends in temperature/CO2). The current model treats samples as independent snapshots.
3.  **Sensor Noise**: Real-world sensors drift. Use of 'Goodness of Fit' tests showed non-normal distributions, validating the use of robust sealers and non-parametric models.

### Future Improvements
1.  **Sensor Fusion**: Integrate weather data (Wind, Humidity) from external APIs.
2.  **Edge Deployment**: Quantize the model (TFLite) to run directly on the drone hardware.
3.  **Time-Series Modeling**: Use LSTMs or GRUs if timestamp data is reliable.


In [ ]:

# Export Artifacts
if 'Fire_Alarm' in df_eng.columns:
    joblib.dump(winner_model, 'fire_detection_model.pkl')
    joblib.dump(scaler, 'feature_scaler.pkl')
    print("[OK] Model and Scaler exported successfully.")
